In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

In [2]:
3. # a rank 0 tensor; a scalar with shape [],
[1., 2., 3.] # a rank 1 tensor; a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]

#Tensors are basically matrices

[[[1.0, 2.0, 3.0]], [[7.0, 8.0, 9.0]]]

You might think of TensorFlow Core programs as consisting of two discrete sections:

1) Building the computational graph (a tf.Graph).

2) Running the computational graph (using a tf.Session).



A Graph contains a set of tf.Operation objects, which represent units of computation; and tf.Tensor objects, which represent the units of data that flow between operations.

A Session object encapsulates the environment in which Operation objects are executed, and Tensor objects are evaluated.

A computational graph is a series of TensorFlow operations arranged into a graph. The graph is composed of two types of objects.

Operations (or "ops"): The nodes of the graph. Operations describe calculations that consume and produce tensors.
Tensors: The edges in the graph. These represent the values that will flow through the graph. Most TensorFlow functions return tf.Tensors.

In [3]:
# building a computational graph, we define graph 
# with tensors a and b, then use and operation of +

a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

# total is an operation of add and produces a tensor add_1:0

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


In [4]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

#This will produce an event file in the current directory with a 
# name in the following format:
# events.out.tfevents.{timestamp}.{hostname}

# Now, in a new terminal, launch TensorBoard with the following 
# shell command: tensorboard --logdir .

# localhost:6006 to open in browser
# SUPER COOL!

To evaluate tensors, instantiate a tf.Session object, informally known as a session. A session encapsulates the state of the TensorFlow runtime, and runs TensorFlow operations. If a tf.Graph is like a .py file, a tf.Session is like the python executable.

In [5]:
sess = tf.Session()
print(sess.run(total))

# When you request the output of a node with Session.run 
# TensorFlow backtracks through the graph and runs 
# all the nodes that provide input to the requested output node. 
# So this prints the expected value of 7.0:

7.0


In [6]:
print(sess.run({'ab':(a, b), 'total':total}))

# can edit what sess run outputs

{'ab': (3.0, 4.0), 'total': 7.0}


In [7]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

# The result shows a different random value on each call to run, 
# but a consistent value during a single run 
# (out1 and out2 receive the same random input):

[0.788303   0.30530667 0.7705107 ]
[0.7839005  0.76697135 0.84752464]
(array([1.3130958, 1.7870046, 1.1058717], dtype=float32), array([2.3130958, 2.7870045, 2.1058717], dtype=float32))


In [8]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

# A placeholder is a promise to provide a value later, 
# like a function argument.

print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

# We can evaluate this graph with multiple inputs by using
# the feed_dict argument of the run method to feed concrete 
# values to the placeholders:


# A way of feeding in multiple things into a session at a time
# or seperately

7.5
[3. 7.]


In [9]:
x = tf.constant(5, dtype=tf.float32)
y = tf.placeholder(tf.float32)
z = x + y
print(sess.run(z, feed_dict={x: 20, y: 4.5}))

# Also note that the feed_dict argument can be used to overwrite 
# any tensor in the graph. The only difference between placeholders 
# and other tf.Tensors is that placeholders throw an error 
# if no value is fed to them.

24.5


Placeholders work for simple experiments, but Datasets are the preferred method of streaming data into a model.

class Dataset: Represents a potentially large set of elements.
class Iterator: Represents the state of iterating through a Dataset.
method make_one_shot_iterator: Creates an Iterator for enumerating the
    elements of this dataset.


In [10]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
# Ttf.Tensor from a Dataset to a tf.data.Iterator
slices = tf.data.Dataset.from_tensor_slices(my_data)
# then call the Iterator's get_next method
next_item = slices.make_one_shot_iterator().get_next()

# calling next_item, or slices.make_one_shot_iterator().get_next(),
# makes the iterator traverse my_data

In [11]:
sess = tf.Session()
print(sess.run(next_item))
print(sess.run(next_item))

[0 1]
[2 3]


In [12]:
sess = tf.Session()
while True:
  try:
    print(sess.run(next_item))
  except tf.errors.OutOfRangeError:
    break
# To iterate til end, or else throws error

[0 1]
[2 3]
[4 5]
[6 7]


In [13]:
# If the Dataset depends on stateful operations you may need to initialize 
# the iterator before using it, as shown below:

r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer) 
#iterator.initializer creates new iterator so can start from the top
while True:
  try:
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    break

[1.2849698  0.39986426 0.8363965 ]
[-0.22005548  0.7852679  -1.3876309 ]
[ 0.9066339 -0.7323097 -0.1944773]
[-0.0963688  -0.2940833  -0.90976125]
[ 0.34254307 -0.5926556   0.83048093]
[-1.2736316   1.5304605  -0.07646449]
[-1.2000003  -1.5103297  -0.77437305]
[ 1.652955    0.84029835 -0.85513127]
[-0.33895183 -0.7121658   0.17647195]
[-0.75605726 -0.49841708  1.1098886 ]


The following code creates a Dense layer that takes a batch of input vectors, and produces a single output value for each. To apply a layer to an input, call the layer as if it were a function. For example:

class Dense: This layer implements the operation: outputs =
    activation(inputs * kernel + bias)
class Layer: Base layer class. This is the class from which all layers 
    inherit, implementing common infrastructure functionality.

In [14]:
x = tf.placeholder(tf.float32, shape=[None, 3]) #example parameters
linear_model = tf.layers.Dense(units=1) #Layer
y = linear_model(x) #Output is layer acting on examples

# The layer contains variables that must be 
# initialized before they can be used. Because they say so

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]})) # applying model on 2 inputs

# tf.layers.dense is a shortcut but bad for debugging

[[-1.1201149]
 [-1.3120627]]


class feature_columns: Think of feature columns as the intermediaries between raw data and Estimators.
function Input column: only accepts dense columns as inputs, Returns a dense Tensor as input layer based on given
    feature_columns.

In [15]:
#Making feature columns

# The easiest way to experiment with feature columns is using 
# the tf.feature_column.input_layer function. 
# This function only accepts dense columns as inputs, 
# so to view the result of a categorical column you must wrap it 
# in an tf.feature_column.indicator_column. For example:

features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']
}

#Defining columns
department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)
#Running the inputs tensor will parse the features into a batch of vectors.

In [16]:
# Feature columns can have internal state, like layers, so they often need to be initialized. 
# Categorical columns use lookup tables internally and these require a separate initialization op, 
# tf.tables_initializer.

var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

# Once the internal state has been initialized you can run inputs like any other tf.Tensor:
print(sess.run(inputs))

# ... hmm to be filled in, unsure how these categories and values are relevant as of yet. 

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


Now that you're familiar with the basics of core TensorFlow, let's train a small regression model manually. HAH!

In [28]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)
y_pred = linear_model(x)  # initalized with 1 layer with 1 input (unit)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred)) # since model not trained output sucks

loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)
print(sess.run(loss))

[[-1.1777302]
 [-2.3554604]
 [-3.5331907]
 [-4.710921 ]]
2.1255612


To optimize a model, you first need to define the loss. We'll use the mean square error, a standard loss for regression problems.

While you could do this manually with lower level math operations, the tf.losses module provides a set of common loss functions. You can use it to calculate the mean square error as follows

TensorFlow provides optimizers implementing standard optimization algorithms. These are implemented as sub-classes of tf.train.Optimizer

In [29]:
optimizer = tf.train.GradientDescentOptimizer(0.01) #input is step size
train = optimizer.minimize(loss) #can use gradient descent after implement loss function in tf

for i in range(10):
  _, loss_value = sess.run((train, loss)) #train doesn't have output so use loss to see learning rate
  print(loss_value)

2.1255612
1.5104313
1.0833931
0.7868695
0.58090764
0.43778658
0.33827022
0.26901168
0.22074969
0.18705788
